# Demonstração: processos cooperando através de filas

### Conceitos: número primo e menor fator primo

Número primo `n` só pode ser representados por `1` fileira de itens.

Números compostos podem ser representados por `f` fileiras de itens.

O '''menor fator primo''' é a menor dimensão que é um número primo.

<img src="prime-cookies.png" width="50%">

### Importar o que vamos usar

In [1]:
import os, math, sys
from time import perf_counter
from typing import NamedTuple, TypeAlias
import multiprocessing as mp
import multiprocessing.queues as mpq

from primes import least_prime_factor, make_sample

from table import Table

### Selecionar a magnitude dos números para trabalhar

Escolha uma magnitute apropriada para que a demonstração não fique lenta demais ou rápida demais na sua máquina.

In [2]:
# may be needed on MacOS 13
# see: https://superfastpython.com/filenotfounderror-multiprocessing-python/
mp.set_start_method('fork')

# Magnitude of primes that take a few seconds to check
#
# machine  magnitude
# RPI4     2 ** 49
# X250     2 ** 53
# YOGA9    2 ** 57
# M2MAX    2 ** 57
# VIVO     2 ** 63

MAGNITUDE = 2**57

print(f'{MAGNITUDE:_}')

144_115_188_075_855_872


In [3]:
sample = sorted(make_sample(MAGNITUDE))

from math import log2

for n in sample:
    print(f'≈ 2↑{log2(n):.0f} {n:24_}')

≈ 2↑54   18_014_398_509_481_951
≈ 2↑54   18_014_398_509_481_984
≈ 2↑54   18_014_399_314_786_597
≈ 2↑55   27_021_597_764_222_939
≈ 2↑55   27_021_597_764_222_976
≈ 2↑55   27_021_598_744_655_129
≈ 2↑55   36_028_797_018_963_913
≈ 2↑55   36_028_797_018_963_968
≈ 2↑55   36_028_803_757_875_637
≈ 2↑56   54_043_195_528_445_869
≈ 2↑56   54_043_195_528_445_952
≈ 2↑56   54_043_196_378_148_947
≈ 2↑56   72_057_594_037_927_931
≈ 2↑56   72_057_594_037_927_936
≈ 2↑56   72_057_596_722_278_677
≈ 2↑57  108_086_391_056_891_903
≈ 2↑57  108_086_391_056_891_904
≈ 2↑57  108_086_392_348_502_419
≈ 2↑57  144_115_188_075_855_859
≈ 2↑57  144_115_188_075_855_872
≈ 2↑57  144_115_189_976_253_901


### Classe que representa um exemplo de fatoração

`n`: inteiro a ser fatorado
`lpf`: Least Prime Factor (menor fator primo)
`elapsed`: tempo transcorrido para fatorar

In [4]:
class Experiment(NamedTuple):  # <3>
    n: int
    lpf: int
    elapsed: float

    @property
    def prime(self):
        return self.n == self.lpf

### Fazer um experimento

In [5]:
def check(n: int) -> Experiment:  # <6>
    t0 = perf_counter()
    res = least_prime_factor(n)
    return Experiment(n, res, perf_counter() - t0)

check(7)

Experiment(n=7, lpf=7, elapsed=2.7909991331398487e-06)

In [6]:
check(9)

Experiment(n=9, lpf=3, elapsed=8.749921107664704e-07)

In [7]:
check(72057596722278677)

Experiment(n=72057596722278677, lpf=268435399, elapsed=2.832699375008815)

In [8]:
72057596722278677 // 268435399

268435523

### Fluxo de trabalho

<img src="process-workflow.png" width="50%">

### Criar um worker

Cad worker é uma função que fica em loop consumindo tarefas da fila: `jobs.get()`.

O resultado de cada `check(n)` processado é um `Experiment`, colocado na fila `results`.

Quando `jobs.get()` devolve `0`, é uma "poison pill" (cápsula de veneno).

Então saímos do laço e colocamos um `Experiment` com `n=0` para sinalizar que terminamos.

In [9]:
JobQueue = mpq.SimpleQueue[int]  # <4>
ResultQueue = mpq.SimpleQueue[Experiment]  # <5>

def worker(jobs: JobQueue, results: ResultQueue) -> None:  # <7>
    while n := jobs.get():  # <8>
        results.put(check(n))  # <9>
    results.put(Experiment(0, False, 0.0))  # <10>

### Enfileirar tarefas

Colocamos cada `n` da amostra na fila `jobs`.

In [10]:
def queue_jobs() -> JobQueue:
    jobs: JobQueue = mp.SimpleQueue()  # <2>
    for n in make_sample(MAGNITUDE):
        jobs.put(n)  # <12>
    return jobs

### Iniciar tarefas

Para cada processo worker criado:

* `proc.start()` inicia o worker;
* `jobs.put(0)` coloca um 0 como sinal para o worker terminar.

In [11]:
def start_jobs(qtd_procs: int, jobs: JobQueue, results: ResultQueue):    
    for _ in range(qtd_procs):
        proc = mp.Process(target=worker, args=(jobs, results))  # <13>
        proc.start()  # <14>
        jobs.put(0)  # <15> "poison pill"

### Exibir resultados

Enquanto ainda há processos executando:

* otemos um `Experiment` da fila `results`;
* se `exp.n` é zero, o processo terminou;
* do contrário, atualizamos a tabela com o resultado.

In [12]:
def report(qtd_procs: int, results: ResultQueue, table: Table) -> int:  # <6>
    checked = 0
    procs_done = 0
    while procs_done < qtd_procs:  # <7>
        exp = results.get()  # <8>
        if exp.n == 0:  # <9>
            procs_done += 1
        else:
            checked += 1  # <10>
            table.update(exp.n, exp.lpf, exp.elapsed)
            
    return checked

### Orquestração dos componentes

* criar e exibir a tabela de resultados com os números da amostra;
* reportar quantidade de processos worker a serem usados;
* registrar momento inicial em `t0`;
* enfileirar tarefas (isso é praticamente imediato);
* criar fila de resultados;
* iniciar tarefas;
* exibir resultados à medida que ficam prontos;
* exibir número de verificações feitas e o tempo transcorrido.


In [13]:
def supervisor(qtd_procs: int) -> None:
    table = Table(sample)
    table.display()
    print(f'Using {qtd_procs} worker processes.')
    t0 = perf_counter()
    jobs = queue_jobs()
    results: ResultQueue = mp.SimpleQueue()
    start_jobs(qtd_procs, jobs, results)  # <3>
    checked = report(qtd_procs, results, table)  # <4>
    elapsed = perf_counter() - t0
    print(f'{checked} checks in {elapsed:.1f}s')

supervisor(os.cpu_count())

Using 12 worker processes.
21 checks in 6.2s
